In [4]:
from tweepy.streaming import StreamListener
from tweepy import OAuthHandler
from tweepy import Stream
import pandas as pd
import json
# import dataset
import credentials
import sqlite3
import nltk
from nltk.tokenize import RegexpTokenizer
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
import pickle
from Naive_bayes_model import *
from sklearn.ensemble import RandomForestClassifier

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\sayali\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\sayali\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\sayali\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [5]:
f = open('nb1.pickle', 'rb')
NBayesClassifier = pickle.load(f)
f.close()

In [6]:
f1 = open('rfp', 'rb')
RFClassifier = pickle.load(f1)
f1.close()

C:\Users\sayali\anaconda3\lib\site-packages\sklearn\utils\deprecation.py:144: FutureWarning: The sklearn.ensemble.forest module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.ensemble. Anything that cannot be imported from sklearn.ensemble is now part of the private API.
  warnings.warn(message, FutureWarning)
C:\Users\sayali\anaconda3\lib\site-packages\sklearn\utils\deprecation.py:144: FutureWarning: The sklearn.tree.tree module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.tree. Anything that cannot be imported from sklearn.tree is now part of the private API.
  warnings.warn(message, FutureWarning)
C:\Users\sayali\anaconda3\lib\site-packages\sklearn\base.py:318: UserWarning: Trying to unpickle estimator DecisionTreeClassifier from version 0.19.1 when using version 0.22.1. This might lead

In [7]:
conn = sqlite3.connect('tweet2.db')
c = conn.cursor()

In [8]:
def pred(data):
    
    tweetProcessor = PreProcessTweets()
    preprocessedTrainingSet = tweetProcessor.processTweets(data)
    
    word_features = buildVocabulary(preprocessedTrainingSet)
#     print("\n",word_features,"\n")
#     buildVocabulary(preprocessedTrainingSet)
    trainingFeatures = nltk.classify.apply_features(extract_features, preprocessedTrainingSet)
    
    print(preprocessedTrainingSet[0][0])
    
    NB = NBayesClassifier.classify(extract_features(preprocessedTrainingSet[0][0],word_features))
    
    return NB
    

In [9]:

def attributeselection(tweet,query_word):
    attr= []

    #text = "Alleged East Bay serial arsonist arrested #SanFrancisco - http://t.co/ojuHfkHVb2"
    tweet = tweet.lower() # convert text to lower-case
    tweet = re.sub('((www\.[^\s]+)|(https?://[^\s]+))', 'URL', tweet) # remove URLs
    tweet = re.sub('@[^\s]+', 'AT_USER', tweet) # remove username
    tweet = re.sub(r'#([^\s]+)', r'\1', tweet) # remove the # in #hashtag
    tweet = re.sub(r'\\s+',r' ', tweet, flags=re.I)
    tweet = re.sub("([^\x00-\x7F])+"," ",tweet) #remove all non-english characters
    tweet = re.sub(r"\s+"," ", tweet, flags = re.I) #remove multiple spaces
    text = tweet
    text = nltk.word_tokenize(text)
    poslist = nltk.pos_tag(text)
#     print(poslist)

    verblist = []
    for i in range(len(poslist)):
        if poslist[i][1].startswith('V'):
            verblist.append(poslist[i])
    
    word_count = len(poslist)
#     print(word_count)

    if len(verblist) == 0:
        l = (query_word,'VB')
        verblist.append(l)
    
#     print(verblist,"\n")
    verb_count = len(verblist)
#     print(verb_count)

   # query_word = 'arson'

    words_before = words_after = 0
    for i in range(len(poslist)):
        if poslist[i][0].startswith(query_word):
            words_before = i

    words_after = word_count - words_before - 1

#     print(words_before)
#     print(words_after)
    verbs = []    
    attr.append(word_count)
#     for i in verblist:
#         print(i[0])
#         verbs.append(i[0])
#     attr.append(verbs)
    attr.append(verb_count)
    attr.append(words_before + 1)
#     attr_query_word.append(query_word)
    attr.append(words_before)
    attr.append(words_after)
    
    
        
    column_names = ['no_of_words', 'no_of_verbs', 'pos_query_word', 'word_before', 'word_after']
    print(attr,column_names)
    df = pd.DataFrame([attr], columns=column_names)
    arr_x=df.to_numpy(dtype=object)
    print(arr_x,arr_x.shape)
    y_pred = RFClassifier.predict(arr_x)
    print("PRIORITY: ",y_pred)
    
    

In [11]:
class TwitterStreamer():
    
    def stream_tweets(self, fetched_tweets_filename, hash_tag_list):
        listener = StdOutListener(fetched_tweets_filename)
        auth = OAuthHandler(credentials.CONSUMER_KEY, credentials.CONSUMER_SECRET)
        auth.set_access_token(credentials.ACCESS_TOKEN, credentials.ACCESS_TOKEN_SECRET)
        
        stream = Stream(auth, listener,tweet_mode='extended')
        
        stream.filter(languages=["en"],track=['coronavirus'])
        
 
class StdOutListener(StreamListener):
    
    def on_status(self, status):
        print(status.text)

    def on_error(self, status):
        print(status)

    def on_data(self, data):
        all_data             = json.loads(data)
        created_at           = all_data['created_at']
        favorite_count       = all_data['favorite_count']
        favorited            = all_data['favorited']
        filter_level         = all_data['filter_level']
        lang                 = all_data['lang']
        retweet_count        = all_data['retweet_count']
        retweeted            = all_data['retweeted']
        source               = all_data['source']
        text                 = all_data['text']
        truncated            = all_data['truncated']
        user_created_at      = all_data['user']['created_at']
        user_followers_count = all_data['user']['followers_count']
        user_location        = all_data['user']['location']
        user_lang            = all_data['user']['lang']
        user_name            = all_data['user']['name']
        user_screen_name     = all_data['user']['screen_name']
        user_time_zone       = all_data['user']['time_zone']
        user_utc_offset      = all_data['user']['utc_offset']
        user_friends_count   = all_data['user']['friends_count']
        geo_enabled          = all_data['user']['geo_enabled']  
        coordinates          = all_data['coordinates']
        geo                  = all_data['geo']
        place                = all_data['place']
        if place is not None:
            place_name       = all_data['place']['name']
        else:
            place_name       = all_data['place']

        print("Tweet: "+text+"\n")
        
        data = [{'text': text,'Classfication': ''}]
        
        info = pred(data)
        
        if info == 'Relevant' or info == 1 or info == '1':
            attributeselection(text,'coronavirus')
            
        print("\n Class: "+info+"\n")
                         
        c.execute('''INSERT INTO tweets 
        (created_at, favorite_count, favorited, filter_level, lang, 
                         retweet_count, retweeted, source, text, truncated, user_created_at,  
                         user_followers_count, user_location, user_lang, user_name, 
                         user_screen_name, user_time_zone, user_friends_count,geo_enabled,coordinates,geo,place_name ) 
            VALUES (?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?)''', 
            (created_at, favorite_count, favorited, filter_level, lang, retweet_count, 
                         retweeted, source, text, truncated, user_created_at, 
                         user_followers_count, user_location, user_lang, user_name, 
                         user_screen_name, user_time_zone, user_friends_count,geo_enabled,coordinates,geo,place_name ))
        
        c.execute('''INSERT INTO tweet_class 
        (text, class) 
            VALUES (?,?)''', 
            (text, info))
        
        conn.commit()

In [12]:
if __name__ == '__main__':
    
    hash_tag_list = ['coronavirus']
    fetched_tweets_filename = "tweets.json"
    
    twitter_streamer = TwitterStreamer()
    twitter_streamer.stream_tweets(fetched_tweets_filename, hash_tag_list)

Tweet: RT @axios: BREAKING: As the Senate debated a $2 trillion coronavirus relief deal, Johns Hopkins data just in shows the death toll surpassed…

['rt', 'breaking', 'senate', 'debated', '2', 'trillion', 'coronavirus', 'relief', 'deal', 'johns', 'hopkins', 'data', 'shows', 'death', 'toll', 'surpassed…']
[26, 2, 13, 12, 13] ['no_of_words', 'no_of_verbs', 'pos_query_word', 'word_before', 'word_after']
[[26 2 13 12 13]] (1, 5)
PRIORITY:  [0]

 Class: 1

Tweet: RT @PatienceAgbabi: There’s one thing #G20 leaders must agree to today: offer debt relief to the world’s poorest countries during #coronavi…

['rt', '’', 'one', 'thing', 'g20', 'leaders', 'must', 'agree', 'today', 'offer', 'debt', 'relief', 'world', '’', 'poorest', 'countries', 'coronavi…']
[24, 2, 1, 0, 23] ['no_of_words', 'no_of_verbs', 'pos_query_word', 'word_before', 'word_after']
[[24 2 1 0 23]] (1, 5)
PRIORITY:  [0]

 Class: 1

Tweet: RT @JoeNBC: Mr. President, @nypost reports that New York hospital workers are being forced 

PRIORITY:  [0]

 Class: 1

Tweet: RT @john_sipher: You can lead a President to intelligence but you can’t make him think. https://t.co/d9yCyTTd1M

['rt', 'lead', 'president', 'intelligence', '’', 'make', 'think']

 Class: 0

Tweet: But isn’t it true ? Sorry, cannot be politically correct here. You have got to call a spade a spade.  It is a virus… https://t.co/ROcV7qIAVG

['’', 'true', 'sorry', 'politically', 'correct', 'got', 'call', 'spade', 'spade', 'virus…']

 Class: 0

Tweet: RT @MarieAnnUK: Sophie in tears this morning when her entire street sang happy birthday to her after her party was called-off due to the Co…

['rt', 'sophie', 'tears', 'morning', 'entire', 'street', 'sang', 'happy', 'birthday', 'party', 'called-off', 'due', 'co…']

 Class: 0

Tweet: RT @jbouie: I am sorry to keep doing this but in World War Z (the book) there is a whole section about how disease and risk experts saw the…

['rt', 'sorry', 'keep', 'world', 'war', 'z', 'book', 'whole', 'section', 'disease', 'risk

PRIORITY:  [0]

 Class: 1

Tweet: RT @Olgachristie: Kevin Maguire @Kevin_Maguire is not happy that Prince Charles was given priority coronovirus testing over NHS staff.  Bes…

['rt', 'kevin', 'maguire', 'happy', 'prince', 'charles', 'given', 'priority', 'coronovirus', 'testing', 'nhs', 'staff', 'bes…']

 Class: 0

Tweet: RT @NoahShachtman: NBC and CNN producers say they're through airing Trump's virus pressers. “We might take it from the top and then cut awa…

['rt', 'nbc', 'cnn', 'producers', 'say', "'re", 'airing', 'trump', "'s", 'virus', 'pressers', '“', 'might', 'take', 'top', 'cut', 'awa…']
[27, 5, 1, 0, 26] ['no_of_words', 'no_of_verbs', 'pos_query_word', 'word_before', 'word_after']
[[27 5 1 0 26]] (1, 5)
PRIORITY:  [0]

 Class: 1

Tweet: All the numbers are going up!  This is so irresponsible of you @realDonaldTrump 
#TrumpVirus 
#DumpTrump2020… https://t.co/81nJyng2fj

['numbers', 'going', 'irresponsible', 'trumpvirus', 'dumptrump2020…']
[17, 4, 1, 0, 16] ['no_of_words', 'no_o

KeyError: 'created_at'